In [1]:
import pandas as pd

In [2]:
def entoak(file_path: str):
    """Loads english to twi dictionary

    Args:
        file_path (str): Path to dictionary file

    Returns:
        df: pd.DataFrame
    """
    df = pd.read_csv(file_path)
    df = df.drop(columns=["pos"])
    return df


In [3]:
??entoak

Signature: entoak(file_path: str)
Source:   
def entoak(file_path: str):
    """Loads english to twi dictionary

    Args:
        file_path (str): Path to dictionary file

    Returns:
        df: pd.DataFrame
    """
    df = pd.read_csv(file_path)
    df = df.drop(columns=["pos"])
    return df
File:      c:\users\os\appdata\local\temp\ipykernel_16168\887595629.py
Type:      function